In [6]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
# behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
# history = pd.read_parquet('train/history.parquet', engine='pyarrow')

# bert = pd.read_parquet('bert_base_multilingual_cased.parquet', engine='pyarrow')
# display = bert['google-bert/bert-base-multilingual-cased']

# Test wikipedia entity embeddings

In [16]:
with open("../../../../glove.840B.300d.txt", "r", encoding="utf8") as f:
    lines = f.readlines()
    print(lines[0]) 

, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.3

In [17]:
entities = []
nr_articles_no_entities = 0
for ner_clusters in articles["ner_clusters"]:
    if len(ner_clusters) == 0:
        nr_articles_no_entities += 1
    for entity in ner_clusters:
        entities.append(entity)
unique_entities = set(entities)
print(f"Number of unique entities: {len(unique_entities)}")
print(f"Percentage of articles without entities: {(nr_articles_no_entities / len(articles)) * 100}%")

Number of unique entities: 44539
Percentage of articles without entities: 26.24650400231459%


In [18]:
found = 0
found_embeddings = []
for line in lines[1:]:
    entity = line.split()[0]
    if entity in unique_entities:
        found += 1
        found_embeddings.append(entity)
print(f"Percentage of entities found in the wikipedia embeddings: {(found/len(unique_entities)  * 100)}%")    

Percentage of entities found in the wikipedia embeddings: 19.20564000089809%


In [19]:
# write to file
with open("../entity/entities2id.txt", "w", encoding="utf8") as f:
    for idx,entity in enumerate(set(found_embeddings)):
        f.write(str(idx+1) +"\t" + entity + "\n")
print(idx)

8548


In [20]:
# test file
with open('../entity/entities2id.txt', encoding="utf8") as f:
    lines = f.readlines()

Entity2Index = {}
Index2Entity = {}
print(len(lines))
for line in lines:
    index, word = line.strip('\n').split('\t')
    if word in Entity2Index or index in Index2Entity:
        raise Exception("Duplicate entity")
    Entity2Index[word] = int(index)
    Index2Entity[int(index)] = word
len(Entity2Index)

8549


8549

In [23]:
doc2entitiesid = {}
for idx, article in articles.iterrows():
    id = article["article_id"]
    doc_entities = article["ner_clusters"]
    for doc_entity in doc_entities:
        if doc_entity not in Entity2Index:
            continue
        if id not in doc2entitiesid:
            doc2entitiesid[id] = []
        doc2entitiesid[id].append(Entity2Index[doc_entity])
            
doc2entitiesid

{'N3042022': [3439, 7022, 3439],
 'N3044020': [2915],
 'N3081381': [4971],
 'N3084181': [8324],
 'N3085367': [5618],
 'N3182586': [6853],
 'N3206668': [8232],
 'N3215889': [867],
 'N3218932': [3740, 2785],
 'N3221177': [1752],
 'N3237286': [6809],
 'N3245987': [785],
 'N3985736': [7004],
 'N3987251': [3133],
 'N4017326': [4335],
 'N4097362': [2826, 3836, 7499, 7499],
 'N4116265': [5612],
 'N4117254': [2363, 4174],
 'N4128375': [387],
 'N4134260': [5191],
 'N4142205': [668],
 'N4160897': [7530],
 'N4179217': [3675],
 'N4181949': [7499],
 'N4182050': [8271],
 'N4195795': [3886],
 'N4201861': [643],
 'N4222067': [7969],
 'N4232214': [7908],
 'N4242477': [4434],
 'N4259198': [3139],
 'N4265498': [5310],
 'N4286079': [3511, 4021],
 'N4288610': [7380],
 'N4305440': [3740, 3740, 2909, 2909],
 'N4312019': [5612, 5612, 2512],
 'N4326682': [2363, 2363],
 'N4344511': [3379],
 'N4345293': [5960],
 'N4403340': [5092],
 'N4411390': [272, 272],
 'N4437767': [3500],
 'N4447027': [4550, 3156, 3656],
 '

In [24]:
# write
with open("../entity/doc2entitiesid.txt", "w", encoding="utf8") as f:
    for doc_id, entities in doc2entitiesid.items():
        f.write(str(doc_id) + "\t" + " ".join([str(entity) for entity in entities]) + "\n")

# test
test_dict = {}
with open('../entity/doc2entitiesid.txt', encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        doc_id, entities = line.strip('\n').split('\t')
        entities = entities.split()
        test_dict[doc_id] = [int(entity) for entity in entities]
test_dict         

{'N3042022': [3439, 7022, 3439],
 'N3044020': [2915],
 'N3081381': [4971],
 'N3084181': [8324],
 'N3085367': [5618],
 'N3182586': [6853],
 'N3206668': [8232],
 'N3215889': [867],
 'N3218932': [3740, 2785],
 'N3221177': [1752],
 'N3237286': [6809],
 'N3245987': [785],
 'N3985736': [7004],
 'N3987251': [3133],
 'N4017326': [4335],
 'N4097362': [2826, 3836, 7499, 7499],
 'N4116265': [5612],
 'N4117254': [2363, 4174],
 'N4128375': [387],
 'N4134260': [5191],
 'N4142205': [668],
 'N4160897': [7530],
 'N4179217': [3675],
 'N4181949': [7499],
 'N4182050': [8271],
 'N4195795': [3886],
 'N4201861': [643],
 'N4222067': [7969],
 'N4232214': [7908],
 'N4242477': [4434],
 'N4259198': [3139],
 'N4265498': [5310],
 'N4286079': [3511, 4021],
 'N4288610': [7380],
 'N4305440': [3740, 3740, 2909, 2909],
 'N4312019': [5612, 5612, 2512],
 'N4326682': [2363, 2363],
 'N4344511': [3379],
 'N4345293': [5960],
 'N4403340': [5092],
 'N4411390': [272, 272],
 'N4437767': [3500],
 'N4447027': [4550, 3156, 3656],
 '

## Image embeddings

In [2]:
import pandas as pd
df = pd.read_parquet('../embedding/image_embeddings.parquet', engine='pyarrow')

In [3]:
df.head()

,article_id,image_embedding
0,9734738,"[-0.023272939, -0.03915197, -0.022547087, 0.04..."
1,8647636,"[-0.04257018, -0.038090054, -0.0064549753, 0.0..."
2,9715678,"[0.012374128, 0.028719611, 0.015983034, -0.007..."
3,4001699,"[-0.026271291, 0.04381648, 0.030391414, 0.0407..."
4,4127160,"[-0.02941335, 0.012621758, 0.0016461773, 0.052..."


## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [9]:
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')

In [10]:
# anchor = trans2tsp('05/08/2023 11:59:59 PM')
# def parse_time_bucket(date):
#     tsp = trans2tsp(date)
#     tsp = tsp - anchor
#     tsp = tsp//(600 * 2.2)
#     return tsp


# Assuming 'impression_time' is already in datetime format, if not convert it
behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

# Set the start date of your data collection period
start_date = pd.to_datetime('2023-05-18')

# Calculate the hour buckets since the start of the period
behaviors['bucket_id'] = ((behaviors['impression_time'] - start_date).dt.total_seconds() // (600)).astype(int)

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains('N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

# Same for article_ids_clicked
if not behaviors['article_ids_clicked'].str.contains('N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].head()

,bucket_id,impression_time,article_ids_inview,article_ids_clicked
0,910,2023-05-24 07:47:53,N9778623 N9778682 N9778669 N9778657 N9778736 N...,N9778657
1,909,2023-05-24 07:33:25,N9778718 N9778728 N9778745 N9778669 N9778657 N...,N9778623
2,906,2023-05-24 07:09:04,N9778657 N9778669 N9772866 N9776259 N9756397 N...,N9778669
3,907,2023-05-24 07:13:14,N9020783 N9778444 N9525589 N7213923 N9777397 N...,N9778628
4,907,2023-05-24 07:11:08,N9777492 N9774568 N9565836 N9335113 N9771223 N...,N9777492


In [11]:
behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].sort_values('bucket_id')

,bucket_id,impression_time,article_ids_inview,article_ids_clicked
167535,42,2023-05-18 07:07:13,N9770997 N9766140 N9771042 N9767697 N9735909,N9771042
108647,42,2023-05-18 07:02:54,N9768583 N9767765 N9746360 N9766042 N9770594 N...,N9770400
108646,42,2023-05-18 07:00:29,N9770798 N9770538 N9770741 N9771042 N9666599 N...,N9770798
108645,42,2023-05-18 07:01:24,N9505214 N9770829 N9758882 N9666599 N9540165 N...,N9770867
91782,42,2023-05-18 07:07:19,N9769404 N9767751 N9770726 N9486080 N9770799 N...,N9769404
...,...,...,...,...
26003,1049,2023-05-25 06:56:04,N9775331 N9678305 N9778155 N9771739 N9776420 N...,N9779737
200991,1049,2023-05-25 06:55:22,N9766434 N9779648 N9780195 N9779713 N9779777 N...,N9779427
25993,1049,2023-05-25 06:54:37,N9779263 N9779577 N9779564 N9779713 N9779205,N9779564
26009,1049,2023-05-25 06:54:04,N9737393 N9776420 N9678305 N9779723 N9778155 N...,N9779797


In [12]:
double_dict = {}

for i, row in behaviors.iterrows():

    for article_id in row["article_ids_inview"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["impressions"] += 1
            
    for article_id in row["article_ids_clicked"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["clicks"] += 1
        
mergeimps = pd.DataFrame(double_dict.values())

In [13]:
# mergeimps.to_excel("mergeimps.xlsx", index=False)
mergeimps.to_csv("../popularity/mergeimps.tsv", sep="\t", index=False, header=False)

In [14]:
# tests
test = mergeimps.groupby("article_id").agg({"clicks": "sum", "impressions": "sum", "bucket_id" :["nunique", "count"]}).reset_index()
test

article_id clicks impressions bucket_id      
                   sum         sum   nunique count
0      N4721905      1          74        29    29
1      N4918926      0           8         2     2
2      N5345489      0          26        18    18
3      N5419013      0           2         2     2
4      N5534630      0           9         5     5
...         ...    ...         ...       ...   ...
3990   N9780162      3          34        14    14
3991   N9780181    136        1458        12    12
3992   N9780193     89        1346        11    11
3993   N9780195     36        1013        10    10
3994   N9780271     11         236         4     4

[3995 rows x 5 columns]

## Create train/val/test.tsv

In [15]:
def process_ids(row):
    clicked_ids = set(row['article_ids_clicked'].split())
    inview_ids = row['article_ids_inview'].split()
    processed_ids = [id for id in inview_ids if id not in clicked_ids]
    return ' '.join(processed_ids)

def write_data(behaviors, history, max_history, data_type):
    # Convert 'impression_time' to string
    behaviors['impression_time'] = behaviors['impression_time'].astype(str)

    # Check if 'impression_time' is already in the desired format
    if not behaviors['impression_time'].str.contains('/').any():
        # Convert 'impression_time' to datetime
        behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

        # Format 'impression_time' in the desired format
        behaviors['impression_time'] = behaviors['impression_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

    # Add 'U' before every 'user_id' if it doesn't already start with 'U'
    behaviors['user_id'] = behaviors['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

    # Convert 'article_ids_inview' to string
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].astype(str)

    # Check if 'article_ids_inview' is already in the desired format
    if not behaviors['article_ids_inview'].str.contains('N').any():
        # For each line in the 'article_ids_inview' column, add 'N' upfront
        # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
        behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

    # Same for article_ids_clicked
    if not behaviors['article_ids_clicked'].str.contains('N').any():
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
        behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

    # Select the required columns
    df = behaviors[['user_id', 'impression_time', 'article_ids_clicked', 'article_ids_inview']]
    df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
    df['article_ids_inview'] = df.apply(process_ids, axis=1)

    if(not isinstance(history['impression_time_fixed'][0], str)):
        history['impression_time_fixed'] = history['impression_time_fixed'].apply(lambda x: x[:max_history] if len(x)>max_history else x)
        history['article_id_fixed'] = history['article_id_fixed'].apply(lambda x: x[:max_history] if len(x)>max_history else x)

    history['impression_time_fixed'] = list(history['impression_time_fixed'])
    # Check if 'impression_time' is already in the desired format
    if not history['impression_time_fixed'].str.contains('/').any():
        # Convert 'impression_time' to datetime
        history['impression_time_fixed'] = [pd.to_datetime(x.astype(str)) for x in history['impression_time_fixed']]

        # Format 'impression_time' in the desired format
        history['impression_time_fixed'] = [x.strftime('%m/%d/%Y %I:%M:%S %p') for x in history['impression_time_fixed']]
        # history['impression_time_fixed'] = list(history['impression_time_fixed'] )

    # Add 'U' before every 'user_id' if it doesn't already start with 'U'
    history['user_id'] = history['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

    # Same for article_ids_clicked
    if not history['article_id_fixed'].str.contains('N').any():
        history['article_id_fixed'] = history['article_id_fixed'].apply(lambda x: 'N' + str(x) )
        history['article_id_fixed'] = history['article_id_fixed'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', ' ').str.replace('\t', ' ').str.replace('  ', ' ')
        # history['article_id_fixed'] = history['article_id_fixed'].str.split(' ')

    # Function to join times and user_ids
    def join_times_and_ids(row):
        # Zip the lists from the 'times' and 'user_ids' columns
        paired_list = zip(row['article_id_fixed'].split(' '), row['impression_time_fixed'])
        # Format each pair and join them into a list
        return [pair for pair in paired_list]

    # Apply the function to each row
    history['combined'] = history.apply(join_times_and_ids, axis=1)

    history = history[['user_id', 'combined']]

    

    # df['tot_clicks'] = df.apply(lambda row: (row['article_ids_clicked'], row['impression_time']),axis=1)
    df['positive'] = df['article_ids_clicked']
    df['negatives'] = df['article_ids_inview'].to_list()
    df['bucket'] = df['impression_time']

    # df_grouped = df.groupby('user_id').agg({'tot_clicks': lambda x: x.tolist()}).reset_index()

    df = df.merge(history[['user_id', 'combined']], on='user_id', how='left')

    docs = df[['user_id', 'combined', 'positive', 'negatives', 'bucket']]

    # Write the DataFrame back to the .tsv file
    if data_type == "train":
        docs.to_csv('../train.tsv', sep='\t', index=False, header=False)
    else:
        split = 0.5
        test_split = docs[:int(len(docs)*split)]
        val_split = docs[int(len(docs)*split):]
        val_split.to_csv('../val.tsv', sep='\t', index=False, header=False)
        test_split.to_csv('../test.tsv', sep='\t', index=False, header=False)



In [ ]:
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')
max_history = 10
write_data(behaviors, history, max_history, "train")

In [ ]:
behaviors = pd.read_parquet('val/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('val/history.parquet', engine='pyarrow')
max_history = 10
write_data(behaviors, history, max_history, "val")

## Create docs.tsv

In [7]:

# Select the required columns
articles['body'] = articles['body'].str.replace('\n', ' ')
articles['title'] = articles['title'].str.replace('\n', ' ')
articles['body'] = articles['body'].str.replace('\t', ' ')
# articles['body'] = articles['body'].fillna('')
# articles['category_str'] = articles['category_str'].fillna('')
articles['topics'] = articles['topics'].astype(str).str.replace('\n', ' ')
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))
docs_file = articles[['article_id', 'title', 'category_str', 'subcategory', 'body', 'topics']]

# Write to a new .tsv file without index
docs_file.to_csv('../docs.tsv', sep='\t', index=False, header=False)

## Create docs_pub_time.tsv

In [8]:
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))

# Convert 'impression_time' to string
articles['published_time'] = articles['published_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not articles['published_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    articles['published_time'] = pd.to_datetime(articles['published_time'])

    # Format 'impression_time' in the desired format
    articles['published_time'] = articles['published_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
pubdocs_file = articles[['article_id', 'published_time']]

# Write to a new .tsv file without index
pubdocs_file.to_csv('../popularity/docs_pub_time.tsv', sep='\t', index=False, header=False)

## Create entities